**Author:** Omer Sakarya \
**Article source:** https://towardsdatascience.com/selenium-tutorial-scraping-glassdoor-com-in-10-minutes-3d0915c6d905 \
**Github Repo:** https://github.com/arapfaik/scraping-glassdoor-selenium

In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import time
import pandas as pd

In [11]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="D:/Projects/Python/Data_science_salary_prediction/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.co.uk/Job/'+keyword+'-jobs-SRCH_KO0,'+str(len(keyword))+'.htm'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(5)

        #Test for the "Sign Up" prompt and get rid of it.
        
        try:            
            driver.find_element_by_class_name("react-job-listing").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(2)
        
        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
        except (NoSuchElementException, StaleElementReferenceException) as e:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("react-job-listing")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:
            
            print(keyword)
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            #job_button.click()  #You might
            driver.execute_script("arguments[0].click();", job_button)
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_class_name("e1tk4kwz1").text
                    location = driver.find_element_by_class_name("e1tk4kwz5").text
                    job_title = driver.find_element_by_class_name("e1tk4kwz4").text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_class_name("e1wijj242").text
            except (NoSuchElementException, StaleElementReferenceException) as e:
                salary_estimate = -1 #You need to set a "not found value. It's important."
                print('salary_estimate')
            
            try:
                rating = driver.find_element_by_class_name("e1tk4kwz2").text
            except (NoSuchElementException, StaleElementReferenceException) as e:
                rating = -1 #You need to set a "not found value. It's important."
                
            #Printing for debugging
            #verbose = True
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                company_tab = driver.find_element_by_xpath('.//div[@data-item="tab" and @data-tab-type="overview"]')
                driver.execute_script("arguments[0].click();", company_tab)
                try:                 
                    size = driver.find_element_by_xpath('.//span[text()="Size"]//following-sibling::*').text
                except (NoSuchElementException, StaleElementReferenceException) as e:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//span[text()="Founded"]//following-sibling::*').text
                except (NoSuchElementException, StaleElementReferenceException) as e:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//span[text()="Type"]//following-sibling::*').text
                except (NoSuchElementException, StaleElementReferenceException) as e:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//span[text()="Industry"]//following-sibling::*').text
                except (NoSuchElementException, StaleElementReferenceException) as e:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//span[text()="Sector"]//following-sibling::*').text
                except (NoSuchElementException, StaleElementReferenceException) as e:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//span[text()="Revenue"]//following-sibling::*').text
                except (NoSuchElementException, StaleElementReferenceException) as e:
                    revenue = -1


            except (NoSuchElementException, StaleElementReferenceException) as e:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            next_page = driver.find_element_by_xpath('.//a[@data-test="pagination-next"]')
            driver.execute_script("arguments[0].click();", next_page)
        except (NoSuchElementException, StaleElementReferenceException) as e:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [13]:
#This line will open a new chrome window and start the scraping.
ds = get_jobs("data-scientist", 500, False)
ds

data-scientist
Progress: 0/500
data-scientist
Progress: 1/500
data-scientist
Progress: 2/500
data-scientist
Progress: 3/500
data-scientist
Progress: 4/500
data-scientist
Progress: 5/500
data-scientist
Progress: 6/500
data-scientist
Progress: 7/500
data-scientist
Progress: 8/500
data-scientist
Progress: 9/500
data-scientist
Progress: 10/500
data-scientist
Progress: 11/500
data-scientist
Progress: 12/500
data-scientist
Progress: 13/500
data-scientist
Progress: 14/500
data-scientist
Progress: 15/500
data-scientist
Progress: 16/500
data-scientist
Progress: 17/500
data-scientist
Progress: 18/500
data-scientist
Progress: 19/500
data-scientist
Progress: 20/500
data-scientist
Progress: 21/500
data-scientist
Progress: 22/500
data-scientist
Progress: 23/500
data-scientist
Progress: 24/500
data-scientist
Progress: 25/500
data-scientist
Progress: 26/500
data-scientist
Progress: 27/500
data-scientist
Progress: 28/500
data-scientist
Progress: 29/500
data-scientist
Progress: 30/500
data-scientist
Pro

data-scientist
Progress: 252/500
data-scientist
Progress: 253/500
data-scientist
Progress: 254/500
data-scientist
Progress: 255/500
data-scientist
Progress: 256/500
data-scientist
Progress: 257/500
data-scientist
Progress: 258/500
data-scientist
Progress: 259/500
data-scientist
Progress: 260/500
data-scientist
Progress: 261/500
data-scientist
Progress: 262/500
data-scientist
Progress: 263/500
data-scientist
Progress: 264/500
data-scientist
Progress: 265/500
data-scientist
Progress: 266/500
data-scientist
Progress: 267/500
data-scientist
Progress: 268/500
data-scientist
Progress: 269/500
data-scientist
Progress: 270/500
data-scientist
Progress: 271/500
data-scientist
Progress: 272/500
data-scientist
Progress: 273/500
data-scientist
Progress: 274/500
data-scientist
Progress: 275/500
data-scientist
Progress: 276/500
data-scientist
Progress: 277/500
data-scientist
Progress: 278/500
data-scientist
Progress: 279/500
data-scientist
Progress: 280/500
data-scientist
Progress: 281/500
data-scien

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,£33K - £39K (Glassdoor Est.),About Us\nWe want to help small businesses win...,3.9,Funding Circle UK\n3.9,"London, England",501 to 1000 Employees,2010,Company - Private,Lending,Finance,$50 to $100 million (USD)
1,Graduate Data Scientist - Modelling,£33K - £39K (Glassdoor Est.),Who are we?\nWe want to help small businesses ...,3.9,Funding Circle UK\n3.9,"London, England",501 to 1000 Employees,2010,Company - Private,Lending,Finance,$50 to $100 million (USD)
2,Graduate Data Scientist - Modelling,£33K - £39K (Glassdoor Est.),Who are we?\nWe want to help small businesses ...,3.9,Funding Circle UK\n3.9,"London, England",501 to 1000 Employees,2010,Company - Private,Lending,Finance,$50 to $100 million (USD)
3,Data Scientist,£33K - £39K (Glassdoor Est.),Data Scientist\nWe are looking for someone tra...,-1,720 Management,"London, England",-1,-1,-1,-1,-1,-1
4,Extended Assurance –Data Analytics and Innovat...,£33K - £39K (Glassdoor Est.),Extended Assurance –Data Analytics and Innovat...,3.8,Ernst & Young\n3.8,"Bristol, England",10000+ Employees,1989,Company - Private,Accounting,Accounting & Legal,$10+ billion (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...
495,Medical Statistician / Real-World Data Scientist,£40K - £60K (Glassdoor Est.),Join the Centre of Excellence for Retrospectiv...,3.9,IQVIA\n3.9,"Leeds, England",10000+ Employees,2017,Company - Public,Biotech & Pharmaceutical,Biotech & Pharmaceutical,$10+ billion (USD)
496,Senior Data Scientist,£40K - £60K (Glassdoor Est.),Job Reference Number: 919-RS-17935004-EX2\n\nJ...,3.5,Public Health England\n3.5,"London, England",5001 to 10000 Employees,2013,Government,National Agencies,Government,$100 to $500 million (USD)
497,Data Science Degree Apprenticeship,£40K - £60K (Glassdoor Est.),DISCLAIMER: Please note this role is not suita...,4.2,Experian\n4.2,"Nottingham, England",10000+ Employees,1980,Company - Public,Financial Analytics & Research,Finance,$2 to $5 billion (USD)
498,Senior Data Scientist (Remote - EU Only),£40K - £60K (Glassdoor Est.),"The Company\nAt MarketFinance, we celebrate am...",3.9,MarketFinance\n3.9,Remote,51 to 200 Employees,2011,Company - Private,Financial Transaction Processing,Finance,$5 to $10 million (USD)


In [15]:
da = get_jobs("data-analyst", 500, False)
da

data-analyst
Progress: 0/500
data-analyst
Progress: 1/500
data-analyst
Progress: 2/500
data-analyst
Progress: 3/500
data-analyst
Progress: 4/500
data-analyst
Progress: 5/500
data-analyst
Progress: 6/500
data-analyst
Progress: 7/500
data-analyst
Progress: 8/500
data-analyst
Progress: 9/500
data-analyst
Progress: 10/500
data-analyst
Progress: 11/500
data-analyst
Progress: 12/500
data-analyst
Progress: 13/500
data-analyst
Progress: 14/500
data-analyst
Progress: 15/500
data-analyst
Progress: 16/500
data-analyst
Progress: 17/500
data-analyst
Progress: 18/500
data-analyst
Progress: 19/500
data-analyst
Progress: 20/500
data-analyst
Progress: 21/500
data-analyst
Progress: 22/500
data-analyst
Progress: 23/500
data-analyst
Progress: 24/500
data-analyst
Progress: 25/500
data-analyst
Progress: 26/500
data-analyst
Progress: 27/500
data-analyst
Progress: 28/500
data-analyst
Progress: 29/500
data-analyst
Progress: 30/500
data-analyst
Progress: 31/500
data-analyst
Progress: 32/500
data-analyst
Progres

data-analyst
Progress: 268/500
data-analyst
Progress: 269/500
data-analyst
Progress: 270/500
data-analyst
Progress: 271/500
data-analyst
Progress: 272/500
data-analyst
Progress: 273/500
data-analyst
Progress: 274/500
data-analyst
Progress: 275/500
data-analyst
Progress: 276/500
data-analyst
Progress: 277/500
data-analyst
Progress: 278/500
data-analyst
Progress: 279/500
data-analyst
Progress: 280/500
data-analyst
Progress: 281/500
data-analyst
Progress: 282/500
data-analyst
Progress: 283/500
data-analyst
Progress: 284/500
data-analyst
Progress: 285/500
data-analyst
Progress: 286/500
data-analyst
Progress: 287/500
data-analyst
Progress: 288/500
data-analyst
Progress: 289/500
data-analyst
Progress: 290/500
data-analyst
Progress: 291/500
data-analyst
Progress: 292/500
data-analyst
Progress: 293/500
data-analyst
Progress: 294/500
data-analyst
Progress: 295/500
data-analyst
Progress: 296/500
data-analyst
Progress: 297/500
data-analyst
Progress: 298/500
data-analyst
Progress: 299/500
data-ana

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Analyst,£33K - £53K (Glassdoor Est.),PURPOSE OF ROLE:\nReporting to the Head of Mon...,4.1,Interactive Investor\n4.1,"Manchester, England",501 to 1000 Employees,1995,Company - Private,Brokerage Service,Finance,$25 to $50 million (USD)
1,Data Business Analyst,£33K - £53K (Glassdoor Est.),Are you a Data Business Analyst with a passion...,3.9,Tata Consultancy Services (TCS)\n3.9,"London, England",10000+ Employees,1968,Company - Public,Computer Hardware & Software,Information Technology,$10+ billion (USD)
2,Market Data Analyst,£33K - £53K (Glassdoor Est.),Market Data Analyst - Prescot\n\nWhat is on of...,-1,Alfred H Knight Group,"Liverpool, England",1001 to 5000 Employees,-1,Company - Private,Mining,Mining & Metals,Unknown / Non-Applicable
3,Data Warehouse Technical Analyst,£33K - £53K (Glassdoor Est.),"To start a career that is out of the ordinary,...",3.6,KANTAR\n3.6,England,10000+ Employees,1993,Company - Private,Research & Development,Business Service,Unknown / Non-Applicable
4,CRO & Data Analyst,£33K - £53K (Glassdoor Est.),We’re on the lookout for a CRO analyst to join...,-1,Hudson Commerce,"Glasgow, Scotland",-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
495,BI Data Analyst -Adobe Analytics - 6 month con...,£33K - £46K (Glassdoor Est.),BI Data Analyst with Adobe Analytics\nSix mont...,4.2,GCS Recruitment Specialists Ltd\n4.2,"London, England",51 to 200 Employees,1991,Company - Private,Staffing & Outsourcing,Business Service,Unknown / Non-Applicable
496,Business Systems & Data Analyst (10 Month FTC),£33K - £46K (Glassdoor Est.),Business Systems & Data Analyst\n10 Month Fixe...,3.6,Baxi Heating\n3.6,"Warwick, West Midlands, England",1001 to 5000 Employees,-1,Company - Private,-1,-1,$100 to $500 million (USD)
497,Data Protection Analyst,£33K - £46K (Glassdoor Est.),Company description\nBoutique consultancy with...,-1,Allendevaux,Remote,-1,-1,-1,-1,-1,-1
498,Data Visualisation Analyst,£33K - £46K (Glassdoor Est.),The Role\n\nLiveScore is establishing an advan...,3.5,LiveScore\n3.5,"London, England",201 to 500 Employees,2019,Company - Private,News Outlet,Media,Unknown / Non-Applicable


In [16]:
de = get_jobs("data-engineer", 500, False)
de

data-engineer
Progress: 0/500
data-engineer
Progress: 1/500
data-engineer
Progress: 2/500
data-engineer
Progress: 3/500
data-engineer
Progress: 4/500
data-engineer
Progress: 5/500
data-engineer
Progress: 6/500
data-engineer
Progress: 7/500
data-engineer
Progress: 8/500
data-engineer
Progress: 9/500
data-engineer
Progress: 10/500
data-engineer
Progress: 11/500
data-engineer
Progress: 12/500
data-engineer
Progress: 13/500
data-engineer
Progress: 14/500
data-engineer
Progress: 15/500
data-engineer
Progress: 16/500
data-engineer
Progress: 17/500
data-engineer
Progress: 18/500
data-engineer
Progress: 19/500
data-engineer
Progress: 20/500
data-engineer
Progress: 21/500
data-engineer
Progress: 22/500
data-engineer
Progress: 23/500
data-engineer
Progress: 24/500
data-engineer
Progress: 25/500
data-engineer
Progress: 26/500
data-engineer
Progress: 27/500
data-engineer
Progress: 28/500
data-engineer
Progress: 29/500
data-engineer
Progress: 30/500
data-engineer
Progress: 31/500
data-engineer
Prog

data-engineer
Progress: 260/500
data-engineer
Progress: 261/500
data-engineer
Progress: 262/500
data-engineer
Progress: 263/500
data-engineer
Progress: 264/500
data-engineer
Progress: 265/500
data-engineer
Progress: 266/500
data-engineer
Progress: 267/500
data-engineer
Progress: 268/500
data-engineer
Progress: 269/500
data-engineer
Progress: 270/500
data-engineer
Progress: 271/500
data-engineer
Progress: 272/500
data-engineer
Progress: 273/500
data-engineer
Progress: 274/500
data-engineer
Progress: 275/500
data-engineer
Progress: 276/500
data-engineer
Progress: 277/500
data-engineer
Progress: 278/500
data-engineer
Progress: 279/500
data-engineer
Progress: 280/500
data-engineer
Progress: 281/500
data-engineer
Progress: 282/500
data-engineer
Progress: 283/500
data-engineer
Progress: 284/500
data-engineer
Progress: 285/500
data-engineer
Progress: 286/500
data-engineer
Progress: 287/500
data-engineer
Progress: 288/500
data-engineer
Progress: 289/500
data-engineer
Progress: 290/500
data-eng

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Lead Data Engineer,£32K - £39K (Glassdoor Est.),What you'll do\nAn opportunity has arisen for ...,4.6,Sky Betting & Gaming\n4.6,"Leeds, England",1001 to 5000 Employees,2000,Company - Public,Internet,Information Technology,$500 million to $1 billion (USD)
1,Data Engineer Virtual Hiring Event,£32K - £39K (Glassdoor Est.),BT Hiring Event - Data Engineer\n\nEvent Detai...,3.8,BT\n3.8,"Birmingham, England",10000+ Employees,1846,Company - Public,Telecommunications Service,Telecommunications,Unknown / Non-Applicable
2,Data Engineer Virtual Hiring Event,£32K - £39K (Glassdoor Est.),BT Hiring Event - Data Engineer\n\nEvent Detai...,3.8,BT\n3.8,"Birmingham, England",10000+ Employees,1846,Company - Public,Telecommunications Service,Telecommunications,Unknown / Non-Applicable
3,Quality Assurance Engineer,£32K - £39K (Glassdoor Est.),Your Job\nThe Enterprise Advertising Experienc...,4.3,Indeed\n4.3,"Aberdeen, Scotland",-1,-1,-1,-1,-1,-1
4,Database Engineer,£32K - £39K (Glassdoor Est.),JOB DESCRIPTION:\nWe’re looking for an experie...,-1,Perch Group,"Blackpool, England",1 to 50 Employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...
495,Big Data Engineer,£37K - £37K (Glassdoor Est.),Experince: hands on Big Data/Hadoop with PySpa...,-1,Diagonal Matrix Ltd,"Reading, England",-1,-1,-1,-1,-1,-1
496,Azure Data Engineer,£37K - £37K (Glassdoor Est.),The role: Azure Data Engineer\nWe are looking ...,3.8,PokerStars\n3.8,"Leeds, England",1001 to 5000 Employees,2001,Company - Public,Internet,Information Technology,$1 to $2 billion (USD)
497,Data Engineer - ETL - Link (Remote),£37K - £37K (Glassdoor Est.),Veeva is a mission-driven organization that as...,4.1,Veeva Systems\n4.1,"Oxford, England",1001 to 5000 Employees,2007,Company - Public,Enterprise Software & Network Solutions,Information Technology,$1 to $2 billion (USD)
498,Data Centre Engineer,£37K - £37K (Glassdoor Est.),"We are UKFast. Over the past two decades, we h...",3.3,UKFast\n3.3,"Manchester, England",201 to 500 Employees,1999,Company - Private,Internet,Information Technology,$50 to $100 million (USD)


In [17]:
df = pd.concat([ds,da,de], ignore_index=True)
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,£33K - £39K (Glassdoor Est.),About Us\nWe want to help small businesses win...,3.9,Funding Circle UK\n3.9,"London, England",501 to 1000 Employees,2010,Company - Private,Lending,Finance,$50 to $100 million (USD)
1,Graduate Data Scientist - Modelling,£33K - £39K (Glassdoor Est.),Who are we?\nWe want to help small businesses ...,3.9,Funding Circle UK\n3.9,"London, England",501 to 1000 Employees,2010,Company - Private,Lending,Finance,$50 to $100 million (USD)
2,Graduate Data Scientist - Modelling,£33K - £39K (Glassdoor Est.),Who are we?\nWe want to help small businesses ...,3.9,Funding Circle UK\n3.9,"London, England",501 to 1000 Employees,2010,Company - Private,Lending,Finance,$50 to $100 million (USD)
3,Data Scientist,£33K - £39K (Glassdoor Est.),Data Scientist\nWe are looking for someone tra...,-1,720 Management,"London, England",-1,-1,-1,-1,-1,-1
4,Extended Assurance –Data Analytics and Innovat...,£33K - £39K (Glassdoor Est.),Extended Assurance –Data Analytics and Innovat...,3.8,Ernst & Young\n3.8,"Bristol, England",10000+ Employees,1989,Company - Private,Accounting,Accounting & Legal,$10+ billion (USD)


In [19]:
df.to_csv('data/data_science_job.csv', index=False)